In [1]:
# STEP 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google'

In [ ]:
# STEP 2: Install Required Libraries
!pip install scikit-image opencv-python


In [ ]:
# STEP 3: Import Libraries
import os
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import joblib
from google.colab import files


In [ ]:
# STEP 4: Fog Removal Function using CLAHE
def remove_fog_clahe(gray_img):
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    return clahe.apply(gray_img)


In [ ]:
def extract_features_from_folder(folder, label):
    features = []
    labels = []
    for img_name in os.listdir(folder):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)              # ✅ Convert to grayscale
            fog_removed = remove_fog_clahe(gray)                      # ✅ Fog removal using CLAHE
            resized = cv2.resize(fog_removed, (128, 128))             # ✅ Resize
            hog_feat = hog(resized, orientations=9, pixels_per_cell=(8, 8),
                           cells_per_block=(2, 2), visualize=False)
            features.append(hog_feat)
            labels.append(label)
    return features, labels


In [ ]:
# STEP 6: Load Data
missile_path = "/content/drive/MyDrive/drdo_project/dataset/missile"
bird_path = "/content/drive/MyDrive/drdo_project/dataset/birds"

missile_features, missile_labels = extract_features_from_folder(missile_path, 1)
bird_features, bird_labels = extract_features_from_folder(bird_path, 0)

X = missile_features + bird_features
y = missile_labels + bird_labels


In [ ]:
# STEP 7: Split Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# STEP 8: Train SVM Model
svm = SVC(kernel='linear')
svm.fit(X_train, y_train)


SVC(kernel='linear')

In [ ]:
# STEP 9: Evaluate Model
y_pred = svm.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print("✅ Classification Report:\n", classification_report(y_test, y_pred))


✅ Accuracy: 0.8931297709923665
✅ Classification Report:
               precision    recall  f1-score   support

           0       0.78      0.65      0.71        79
           1       0.91      0.96      0.93       314

    accuracy                           0.89       393
   macro avg       0.85      0.80      0.82       393
weighted avg       0.89      0.89      0.89       393



In [ ]:
# STEP 10: Save Model
joblib.dump(svm, "svm_model.pkl")
files.download("svm_model.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# STEP 11: Predict Single Image with Fog Removal
def predict_image(image_path):
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ Error: Unable to load image from {image_path}")
        return None  # Return None or handle the error appropriately

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fog_removed = remove_fog_clahe(gray)
    resized = cv2.resize(fog_removed, (128, 128))
    features = hog(resized, orientations=9, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
    prediction = svm.predict([features])[0]
    label = "Missile" if prediction == 1 else "Bird"
    print("🔍 Prediction:", label)
    return label


In [ ]:
test_folder = "/content/0bfde743b0958b32_jpg.rf.4034be3246229be078444bf092485154.jpg"  # ⬅️ CHANGE ONLY THIS
predict_image(test_folder)

🔍 Prediction: Missile


'Missile'